# ETL: Silver → Gold

## Contexto do Projeto

Este notebook realiza o processo de ETL (Extract, Transform, Load) para transformar dados da camada Silver para a camada Gold em um pipeline de dados. A camada Gold é destinada a análises avançadas e relatórios, contendo dados limpos, agregados e otimizados para consultas.

## Processo ETL
1. **Extração (Extract)**:
   - Os dados são extraídos da tabela Silver, que contém dados semi-estruturados e pré-processados. 

2. **Transformação (Transform)**:
    - Cria dimensões e fatos a partir dos dados extraídos (Star Schema).

3. **Carga (Load)**:
   - Os dados transformados são carregados no schema Gold.

## 1. Configuração e Imports

In [30]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import psycopg2
from psycopg2.extras import execute_batch
import warnings

warnings.filterwarnings('ignore')

In [31]:
# Configuração de conexão com o banco de dados PostgreSQL
DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': os.getenv('DB_PORT', '5433'),
    'database': os.getenv('DB_NAME', 'grupo08'),
    'user': os.getenv('DB_USER', 'postgres'),
    'password': os.getenv('DB_PASSWORD', 'postgres')
}

## 2. Extração de Dados da Camada Silver

In [32]:
# Estabelecendo a conexão com o banco de dados
try:
    conn = psycopg2.connect(**DB_CONFIG)
    print("Conexão com o banco de dados estabelecida com sucesso.")
except Exception as e:
    print(f"Erro ao conectar ao banco de dados: {e}")

# Extração de Dados da Camada Silver
silver_table = "silver.filmes"
try:
    df = pd.read_sql_query(f"SELECT * FROM {silver_table};", conn)
    print("Dados extraídos com sucesso.")
    print(f"Número de registros extraídos: {len(df)}")
    print(f"Número de colunas extraídas: {len(df.columns)}")
except Exception as e:
    print(f"Erro ao extrair dados: {e}")

Conexão com o banco de dados estabelecida com sucesso.
Dados extraídos com sucesso.
Número de registros extraídos: 1111347
Número de colunas extraídas: 29


## 3. Transformação dos Dados e Carga na Camada Gold

Nesta etapa, será criado o Star Schema contendo as tabelas de dimensão e fato. Utilizaremos Surrogate Keys (SK) e aplicaremos o padrão mnemônico na nomenclatura das colunas.

In [48]:
# Criando schema Gold se não existir
ddl_schema = "CREATE SCHEMA IF NOT EXISTS gold;"

try:
    with conn.cursor() as cursor:
        cursor.execute(ddl_schema)
        conn.commit()
    print("Schema 'gold' verificado/criado com sucesso.")
except Exception as e:
    print(f"Erro ao criar/verificar schema 'gold': {e}")

Schema 'gold' verificado/criado com sucesso.


In [49]:
print ("Criando as tableas fato e dimensão na camada Gold...")

# 1. DIMENSÃO RELEASE (CALENDÁRIO)
try:
    with conn.cursor() as cur:
        print("Criando dimensão Release...")
        
        sql_dim_release = """
        DROP TABLE IF EXISTS gold.dim_release;

        CREATE TABLE gold.dim_release AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY release_date) AS srk_rel,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            release_date          AS rel_dat,
            release_year          AS rel_yer,
            release_month         AS rel_mon,
            release_month_name    AS rel_mon_nam,
            release_day_of_week   AS rel_day_wek,
            release_day_name      AS rel_day_nam,
            release_decade        AS rel_dec
            
        FROM (
            -- Seleciona apenas datas únicas existentes na tabela de filmes
            SELECT DISTINCT release_date, release_year, release_month, release_month_name,
                            release_day_of_week, release_day_name, release_decade
            FROM silver.filmes 
            WHERE release_date IS NOT NULL
        ) sub;

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_release ADD PRIMARY KEY (srk_rel);
        """
        cur.execute(sql_dim_release)
        print("gold.dim_release criada com sucesso.")

        df_dim_release = pd.read_sql_query("SELECT * FROM gold.dim_release;", conn)
        print(f"Número de registros em gold.dim_release: {len(df_dim_release)}")
        print(f"Número de colunas em gold.dim_release: {len(df_dim_release.columns)}")
        print(df_dim_release.head())
    
except Exception as e:
    print(f"Erro ao criar gold.dim_release: {e}")

finally:
    conn.commit()

Criando as tableas fato e dimensão na camada Gold...
Criando dimensão Release...
Erro ao criar gold.dim_release: cannot drop table gold.dim_release because other objects depend on it
DETAIL:  constraint frk_rel on table gold.fact_movie depends on table gold.dim_release
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [50]:
# 2. DIMENSÃO GÊNERO
try:
    print ("Criando dimensão Genre...")
    with conn.cursor() as cur:
        
        sql_dim_genre = """
        DROP TABLE IF EXISTS gold.dim_genre;

        CREATE TABLE gold.dim_genre AS
        WITH uniao_generos AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(genres, ','))) AS nm_genre
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_genre) AS nm_genre
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_genre) AS srk_gen,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_genre AS gen
            
        FROM uniao_generos
        WHERE nm_genre IS NOT NULL AND nm_genre <> '';

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_genre ADD PRIMARY KEY (srk_gen);
        """
        
        cur.execute(sql_dim_genre)
        conn.commit()
        print("gold.dim_genre criada com sucesso.")

        df_dim_genre = pd.read_sql_query("SELECT * FROM gold.dim_genre;", conn)
        print(f"Número de registros em gold.dim_genre: {len(df_dim_genre)}")
        print(f"Número de colunas em gold.dim_genre: {len(df_dim_genre.columns)}")
        print(df_dim_genre.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Genre...
Erro na execução: cannot drop table gold.dim_genre because other objects depend on it
DETAIL:  constraint frk_gen on table gold.fact_movie depends on table gold.dim_genre
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [51]:
# 3. DIMENSÃO LUCRO

try:
    with conn.cursor() as cur:
        print("Criando dimensão Profit...")
        
        sql_dim_profit = """
        DROP TABLE IF EXISTS gold.dim_profit;

        CREATE TABLE gold.dim_profit AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY budget, revenue) AS srk_pft,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            budget          AS bdg,
            revenue         AS rev,
            profit          AS pft,
            roi             AS ret_inv,
            is_profitable   AS pfe,
            budget_tier     AS bdg_tir

        FROM (
            -- Seleciona combinações únicas de valores financeiros
            SELECT DISTINCT 
                budget, revenue, profit, roi, is_profitable, budget_tier
            FROM silver.filmes
        ) sub;

        ALTER TABLE gold.dim_profit ADD PRIMARY KEY (srk_pft);
        """
        cur.execute(sql_dim_profit)
        print("gold.dim_profit criada com sucesso.")

        df_dim_profit = pd.read_sql_query("SELECT * FROM gold.dim_profit;", conn)
        print(f"Número de registros em gold.dim_profit: {len(df_dim_profit)}")
        print(f"Número de colunas em gold.dim_profit: {len(df_dim_profit.columns)}")
        print(df_dim_profit.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()


Criando dimensão Profit...
Erro na execução: cannot drop table gold.dim_profit because other objects depend on it
DETAIL:  constraint frk_pft on table gold.fact_movie depends on table gold.dim_profit
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [52]:
# 4. DIMENSÃO ENGAGEMENT
try:
    with conn.cursor() as cur:
        print("Criando dimensão Engagement...")
        
        sql_dim_engagement = """
        DROP TABLE IF EXISTS gold.dim_engagement;

        CREATE TABLE gold.dim_engagement AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY popularity DESC) AS srk_eng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            vote_average    AS vot_avg,
            vote_count      AS vot_cnt,
            popularity      AS pop

        FROM (
            -- Seleciona combinações únicas de engajamento
            SELECT DISTINCT 
                vote_average, vote_count, popularity
            FROM silver.filmes
        ) sub;

        ALTER TABLE gold.dim_engagement ADD PRIMARY KEY (srk_eng);
        """
        cur.execute(sql_dim_engagement)
        print("gold.dim_engagement criada com sucesso.")

        df_dim_engagement = pd.read_sql_query("SELECT * FROM gold.dim_engagement;", conn)
        print(f"Número de registros em gold.dim_engagement: {len(df_dim_engagement)}")
        print(f"Número de colunas em gold.dim_engagement: {len(df_dim_engagement.columns)}")
        print(df_dim_engagement.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Engagement...
Erro na execução: cannot drop table gold.dim_engagement because other objects depend on it
DETAIL:  constraint frk_eng on table gold.fact_movie depends on table gold.dim_engagement
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [53]:
# 5. Dimensão Language

try:
    with conn.cursor() as cur:
        print("Criando dimensão Language...")
        
        sql_dim_language = """
        DROP TABLE IF EXISTS gold.dim_language;

        CREATE TABLE gold.dim_language AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY original_language) AS srk_lng,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            original_language AS lng

        FROM (
            -- Seleciona línguas únicas
            SELECT DISTINCT original_language
            FROM silver.filmes
            WHERE original_language IS NOT NULL AND original_language <> ''
        ) sub;

        ALTER TABLE gold.dim_language ADD PRIMARY KEY (srk_lng);
        """
        cur.execute(sql_dim_language)
        print("gold.dim_language criada com sucesso.")

        df_dim_language = pd.read_sql_query("SELECT * FROM gold.dim_language;", conn)
        print(f"Número de registros em gold.dim_language: {len(df_dim_language)}")
        print(f"Número de colunas em gold.dim_language: {len(df_dim_language.columns)}")
        print(df_dim_language.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Language...
Erro na execução: cannot drop table gold.dim_language because other objects depend on it
DETAIL:  constraint frk_lng on table gold.fact_movie depends on table gold.dim_language
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [54]:
# 6. DIMENSÃO COMPANY
try:
    print ("Criando dimensão Company...")
    with conn.cursor() as cur:
        
        sql_dim_company = """
        DROP TABLE IF EXISTS gold.dim_company;

        CREATE TABLE gold.dim_company AS
        WITH uniao_companies AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_companies, ','))) AS nm_company
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_company) AS nm_company
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_company) AS srk_cmp,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_company AS prd_cmp
            
        FROM uniao_companies
        WHERE nm_company IS NOT NULL AND nm_company <> '';

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_company ADD PRIMARY KEY (srk_cmp);
        """
        
        cur.execute(sql_dim_company)
        conn.commit()
        print("gold.dim_company criada com sucesso.")
        df_dim_company = pd.read_sql_query("SELECT * FROM gold.dim_company;", conn)
        print(f"Número de registros em gold.dim_company: {len(df_dim_company)}")
        print(f"Número de colunas em gold.dim_company: {len(df_dim_company.columns)}")
        print(df_dim_company.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Company...
Erro na execução: cannot drop table gold.dim_company because other objects depend on it
DETAIL:  constraint frk_cmp on table gold.fact_movie depends on table gold.dim_company
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [55]:
# 7. DIMENSÃO COUNTRY
try:
    print ("Criando dimensão Country...")
    with conn.cursor() as cur:
        
        sql_dim_country = """
        DROP TABLE IF EXISTS gold.dim_country;
        CREATE TABLE gold.dim_country AS
        WITH uniao_countries AS (
            SELECT 
                TRIM(UNNEST(STRING_TO_ARRAY(production_countries, ','))) AS nm_country
            FROM silver.filmes
            
            UNION
            
            SELECT 
                TRIM(primary_country) AS nm_country
            FROM silver.filmes
        )
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY nm_country) AS srk_ctr,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            nm_country AS prd_ctr
            
        FROM uniao_countries
        WHERE nm_country IS NOT NULL AND nm_country <> '';

        -- Definindo a chave primária física
        ALTER TABLE gold.dim_country ADD PRIMARY KEY (srk_ctr);
        """
        
        cur.execute(sql_dim_country)
        conn.commit()
        print("gold.dim_country criada com sucesso.")
        df_dim_country = pd.read_sql_query("SELECT * FROM gold.dim_country;", conn)
        print(f"Número de registros em gold.dim_country: {len(df_dim_country)}")
        print(f"Número de colunas em gold.dim_country: {len(df_dim_country.columns)}")
        print(df_dim_country.head())
except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Country...
Erro na execução: cannot drop table gold.dim_country because other objects depend on it
DETAIL:  constraint frk_cnt on table gold.fact_movie depends on table gold.dim_country
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [56]:
# 8. DIMENSÃO RUNTIME

try:
    with conn.cursor() as cur:
        print("Criando dimensão Runtime...")
        
        sql_dim_runtime = """
        DROP TABLE IF EXISTS gold.dim_runtime;
        CREATE TABLE gold.dim_runtime AS
        SELECT
            -- Criação da Surrogate Key (SRK)
            ROW_NUMBER() OVER (ORDER BY runtime) AS srk_rte,
            
            -- Renomeação e Extração (Padrão Mnemônico)
            runtime AS rte

        FROM (
            -- Seleciona runtime únicos
            SELECT DISTINCT runtime
            FROM silver.filmes
            WHERE runtime IS NOT NULL
        ) sub;

        ALTER TABLE gold.dim_runtime ADD PRIMARY KEY (srk_rte);
        """
        cur.execute(sql_dim_runtime)
        print("gold.dim_runtime criada com sucesso.")

        df_dim_runtime = pd.read_sql_query("SELECT * FROM gold.dim_runtime;", conn)
        print(f"Número de registros em gold.dim_runtime: {len(df_dim_runtime)}")
        print(f"Número de colunas em gold.dim_runtime: {len(df_dim_runtime.columns)}")
        print(df_dim_runtime.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução: {e}")

finally:
    conn.commit()

Criando dimensão Runtime...
Erro na execução: cannot drop table gold.dim_runtime because other objects depend on it
DETAIL:  constraint frk_rte on table gold.fact_movie depends on table gold.dim_runtime
HINT:  Use DROP ... CASCADE to drop the dependent objects too.



In [57]:
# 9. FATO FILME

try:
    with conn.cursor() as cur:
        print("Criando tabela fato filme...")
        
        # Etapa 1: Criar a Tabela com os Joins corretos
        sql_fact_movie = """
        DROP TABLE IF EXISTS gold.fact_movie;

        CREATE TABLE gold.fact_movie AS
        SELECT
            -- Criação da Surrogate Key do Fato (SRK)
            ROW_NUMBER() OVER (ORDER BY f.id) AS srk_ttl,
            
            -- Atributos do Filme
            f.title                   AS ttl,
            f.original_title          AS org_ttl,
            f.created_at              AS crt,
            
            -- Chaves Estrangeiras
            dr.srk_rel,  -- Vem de dim_release (dr)
            dg.srk_gen,  -- Vem de dim_genre (dg)
            dp.srk_pft,  -- Vem de dim_profit (dp)
            de.srk_eng,  -- Vem de dim_engagement (de)
            dl.srk_lng,  -- Vem de dim_language (dl)
            dc.srk_cmp,  -- Vem de dim_company (dc)
            dct.srk_ctr, -- Vem de dim_country (dct)
            drt.srk_rte  -- Vem de dim_runtime (drt)

        FROM silver.filmes f
        
        -- 1. Dimensão Release (Alias: dr)
        LEFT JOIN gold.dim_release dr ON f.release_date = dr.rel_dat
        
        -- 2. Dimensão Genre (Alias: dg)
        LEFT JOIN gold.dim_genre dg ON TRIM(f.primary_genre) = dg.gen
        
        -- 3. Dimensão Profit (Alias: dp)
        LEFT JOIN gold.dim_profit dp ON 
            f.budget = dp.bdg AND 
            f.revenue = dp.rev AND
            f.profit = dp.pft
            
        -- 4. Dimensão Engagement (Alias: de)
        LEFT JOIN gold.dim_engagement de ON 
            f.popularity = de.pop AND
            f.vote_count = de.vot_cnt
            
        -- 5. Dimensão Language (Alias: dl)
        LEFT JOIN gold.dim_language dl ON f.original_language = dl.lng
        
        -- 6. Dimensão Company (Alias: dc)
        LEFT JOIN gold.dim_company dc ON TRIM(f.primary_company) = dc.prd_cmp
        
        -- 7. Dimensão Country (Alias: dct)
        LEFT JOIN gold.dim_country dct ON TRIM(f.primary_country) = dct.prd_ctr
        
        -- 8. Dimensão Runtime (Alias: drt)
        LEFT JOIN gold.dim_runtime drt ON f.runtime = drt.rte;
        """
        cur.execute(sql_fact_movie)
        print("Tabela gold.fact_movie criada (dados inseridos).")

        # Adicionar Chaves Primárias e Estrangeiras
        print("Aplicando chaves e restrições...")
        sql_constraints = """
        ALTER TABLE gold.fact_movie ADD PRIMARY KEY (srk_ttl);
        
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_rel FOREIGN KEY (srk_rel) REFERENCES gold.dim_release(srk_rel);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_gen FOREIGN KEY (srk_gen) REFERENCES gold.dim_genre(srk_gen);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_pft FOREIGN KEY (srk_pft) REFERENCES gold.dim_profit(srk_pft);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_eng FOREIGN KEY (srk_eng) REFERENCES gold.dim_engagement(srk_eng);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_lng FOREIGN KEY (srk_lng) REFERENCES gold.dim_language(srk_lng);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_cmp FOREIGN KEY (srk_cmp) REFERENCES gold.dim_company(srk_cmp);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_cnt FOREIGN KEY (srk_ctr) REFERENCES gold.dim_country(srk_ctr);
        ALTER TABLE gold.fact_movie ADD CONSTRAINT frk_rte FOREIGN KEY (srk_rte) REFERENCES gold.dim_runtime(srk_rte);
        """
        cur.execute(sql_constraints)
        print("Chaves aplicadas com sucesso.")

        # Validação
        df_fact_movie = pd.read_sql_query("SELECT * FROM gold.fact_movie LIMIT 5;", conn)
        print(f"Número de registros inseridos: {len(df_fact_movie)}")
        print(f"Colunas: {list(df_fact_movie.columns)}")
        print(df_fact_movie.head())

except Exception as e:
    conn.rollback()
    print(f"Erro na execução da Fato: {e}")

finally:
    conn.commit()
    conn.close()

Criando tabela fato filme...
Tabela gold.fact_movie criada (dados inseridos).
Aplicando chaves e restrições...
Chaves aplicadas com sucesso.
Número de registros inseridos: 5
Colunas: ['srk_ttl', 'ttl', 'org_ttl', 'crt', 'srk_rel', 'srk_gen', 'srk_pft', 'srk_eng', 'srk_lng', 'srk_cmp', 'srk_ctr', 'srk_rte']
   srk_ttl                               ttl  \
0        1                             Ariel   
1        2               Shadows in Paradise   
2        3                        Four Rooms   
3        4                    Judgment Night   
4        5  Life in Loops (A Megacities RMX)   

                            org_ttl                        crt  srk_rel  \
0                             Ariel 2026-01-24 17:44:09.128896    24171   
1              Varjoja paratiisissa 2026-01-24 17:44:09.128896    23436   
2                        Four Rooms 2026-01-24 17:44:09.128896    26776   
3                    Judgment Night 2026-01-24 17:44:09.128896    25991   
4  Life in Loops (A Megaciti